# Jour 2 - Exercice 2 : Premiers modèles de Machine Learning

## Objectifs
- Comprendre les bases des algorithmes de classification
- Implémenter et évaluer différents modèles de classification
- Analyser les performances des modèles
- Visualiser les résultats avec Plotly
- Comprendre les forces et faiblesses de chaque algorithme

## Introduction

Dans ce notebook, nous allons utiliser les données prétraitées du notebook précédent pour entraîner nos premiers modèles de machine learning. Nous nous concentrerons sur la classification binaire pour prédire si un passager est satisfait ou non de son vol. Nous explorerons plusieurs algorithmes de classification et comparerons leurs performances.

## 1. Chargement et préparation des données

In [1]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Bibliothèques pour le machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Métriques d'évaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc

# Pour afficher plus de colonnes dans les DataFrames
pd.set_option('display.max_columns', 30)

In [ ]:
# Chargement du jeu de données
df = pd.read_csv('../../data/passenger_satisfaction/train.csv')

# Affichage des premières lignes
df.head()

In [ ]:
# Vérification de la distribution de la variable cible
target_counts = df['Satisfaction'].value_counts()

fig = px.pie(values=target_counts.values, 
             names=target_counts.index, 
             title='Distribution de la satisfaction des passagers',
             color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_traces(textinfo='percent+label')
fig.show()

## 2. Prétraitement des données

Nous allons reprendre les étapes de prétraitement du notebook précédent pour préparer nos données pour le machine learning.

In [ ]:
# Traitement des valeurs manquantes
missing_values = df.isnull().sum()
print("Valeurs manquantes par colonne:")
print(missing_values[missing_values > 0])

In [ ]:
# Encodage de la variable cible
le = LabelEncoder()
df['Satisfaction_encoded'] = le.fit_transform(df['Satisfaction'])
print(f"Classes encodées: {dict(zip(le.classes_, range(len(le.classes_))))}")

## 3. Création du pipeline de prétraitement

Nous allons créer un pipeline de prétraitement pour automatiser la transformation des données.

In [ ]:
# Identification des types de colonnes
# Suppression des colonnes ID et Satisfaction (variable cible)
df_features = df.drop(['ID', 'Satisfaction', 'Satisfaction_encoded'], axis=1)

# Identification des colonnes numériques et catégorielles
numeric_features = df_features.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df_features.select_dtypes(include=['object']).columns.tolist()

print(f"Caractéristiques numériques: {numeric_features}")
print(f"Caractéristiques catégorielles: {categorical_features}")

In [ ]:
# Création du pipeline de prétraitement
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Imputation des valeurs manquantes
    ('scaler', StandardScaler())  # Standardisation des données numériques
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputation des valeurs manquantes
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))  # One-hot encoding
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Création de X (features) et y (variable cible)
X = df.drop(['ID', 'Satisfaction', 'Satisfaction_encoded'], axis=1)
y = df['Satisfaction_encoded']

# Division des données en ensembles d'entraînement et de test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Taille de l'ensemble d'entraînement: {X_train.shape}")
print(f"Taille de l'ensemble de validation: {X_val.shape}")

In [ ]:
# Application du prétraitement
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

print(f"Forme des données d'entraînement prétraitées: {X_train_preprocessed.shape}")
print(f"Forme des données de validation prétraitées: {X_val_preprocessed.shape}")

## 4. Fonction d'évaluation des modèles

Nous allons créer une fonction pour évaluer les performances des différents modèles de manière cohérente.

In [9]:
def evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions sur l'ensemble d'entraînement et de validation
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Calcul des métriques
    train_accuracy = accuracy_score(y_train, y_train_pred)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    
    train_precision = precision_score(y_train, y_train_pred)
    val_precision = precision_score(y_val, y_val_pred)
    
    train_recall = recall_score(y_train, y_train_pred)
    val_recall = recall_score(y_val, y_val_pred)
    
    train_f1 = f1_score(y_train, y_train_pred)
    val_f1 = f1_score(y_val, y_val_pred)
    
    # Affichage des métriques
    print(f"Performances du modèle {model_name}:")
    print(f"Accuracy - Train: {train_accuracy:.4f}, Validation: {val_accuracy:.4f}")
    print(f"Precision - Train: {train_precision:.4f}, Validation: {val_precision:.4f}")
    print(f"Recall - Train: {train_recall:.4f}, Validation: {val_recall:.4f}")
    print(f"F1 Score - Train: {train_f1:.4f}, Validation: {val_f1:.4f}")
    print("\nRapport de classification sur l'ensemble de validation:")
    print(classification_report(y_val, y_val_pred))
    
    # Matrice de confusion
    cm = confusion_matrix(y_val, y_val_pred)
    
    # Visualisation de la matrice de confusion avec Plotly
    fig = px.imshow(cm, 
                    labels=dict(x="Prédiction", y="Réalité", color="Nombre"),
                    x=['Non satisfait', 'Satisfait'],
                    y=['Non satisfait', 'Satisfait'],
                    text_auto=True,
                    title=f"Matrice de confusion - {model_name}",
                    color_continuous_scale='Blues')
    fig.update_layout(
        title_text=f"Matrice de confusion - {model_name}",
        xaxis_title="Prédiction",
        yaxis_title="Réalité",
        coloraxis_colorbar=dict(title="Nombre"),
        height=800,
        width=800
    )
    fig.show()
    
    # Courbe ROC si le modèle peut prédire des probabilités
    if hasattr(model, "predict_proba"):
        y_val_proba = model.predict_proba(X_val)[:, 1]
        fpr, tpr, _ = roc_curve(y_val, y_val_proba)
        roc_auc = auc(fpr, tpr)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'{model_name} (AUC = {roc_auc:.4f})'))
        fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Aléatoire', line=dict(dash='dash', color='gray')))
        
        fig.update_layout(
            title=f'Courbe ROC - {model_name}',
            xaxis_title='Taux de faux positifs',
            yaxis_title='Taux de vrais positifs',
            legend=dict(x=0.7, y=0.1),
            width=1000,
            height=800
        )
        fig.show()
    
    return {
        'model': model,
        'name': model_name,
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy,
        'train_precision': train_precision,
        'val_precision': val_precision,
        'train_recall': train_recall,
        'val_recall': val_recall,
        'train_f1': train_f1,
        'val_f1': val_f1
    }

## 5. Implémentation des modèles de classification

Nous allons implémenter plusieurs modèles de classification et évaluer leurs performances.

### 5.1 Modèle de base : Dummy Classifier

Commençons par un modèle de base simple pour établir une référence de performance.

In [ ]:
from sklearn.dummy import DummyClassifier

# Création du modèle de base (stratégie la plus fréquente)
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=42)

# Évaluation du modèle
dummy_results = evaluate_model(dummy_clf, X_train_preprocessed, X_val_preprocessed, y_train, y_val, "Dummy Classifier")

### 5.2 Régression Logistique

La régression logistique est un algorithme simple mais efficace pour la classification binaire.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Création du modèle de régression logistique
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Évaluation du modèle
log_reg_results = evaluate_model(log_reg, X_train_preprocessed, X_val_preprocessed, y_train, y_val, "Régression Logistique")

### 5.3 Arbre de décision

Les arbres de décision sont des modèles intuitifs qui peuvent capturer des relations non linéaires dans les données.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Création du modèle d'arbre de décision
dt_clf = DecisionTreeClassifier(random_state=42)

# Évaluation du modèle
dt_results = evaluate_model(dt_clf, X_train_preprocessed, X_val_preprocessed, y_train, y_val, "Arbre de Décision")

### 5.4 Random Forest

Random Forest est un ensemble d'arbres de décision qui peut améliorer les performances par rapport à un seul arbre.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Création du modèle Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Évaluation du modèle
rf_results = evaluate_model(rf_clf, X_train_preprocessed, X_val_preprocessed, y_train, y_val, "Random Forest")

## 6. Comparaison des modèles

Comparons maintenant les performances des différents modèles.

In [ ]:
# Création d'un DataFrame pour comparer les modèles
results = [dummy_results, log_reg_results, dt_results, rf_results]
comparison_df = pd.DataFrame({
    'Modèle': [result['name'] for result in results],
    'Accuracy (Train)': [result['train_accuracy'] for result in results],
    'Accuracy (Validation)': [result['val_accuracy'] for result in results],
    'Precision (Validation)': [result['val_precision'] for result in results],
    'Recall (Validation)': [result['val_recall'] for result in results],
    'F1 Score (Validation)': [result['val_f1'] for result in results]
})

comparison_df

In [ ]:
# Visualisation des performances des modèles
metrics = ['Accuracy (Validation)', 'Precision (Validation)', 'Recall (Validation)', 'F1 Score (Validation)']
model_names = comparison_df['Modèle']

fig = go.Figure()

for metric in metrics:
    fig.add_trace(go.Bar(
        x=model_names,
        y=comparison_df[metric],
        name=metric
    ))

fig.update_layout(
    title='Comparaison des performances des modèles',
    xaxis_title='Modèle',
    yaxis_title='Score',
    barmode='group',
    width=900,
    height=500
)

fig.show()

## 7. Analyse des caractéristiques importantes

Analysons les caractéristiques les plus importantes pour la prédiction selon le modèle Random Forest, qui a généralement de bonnes performances.

In [ ]:
# Obtention des noms des caractéristiques après transformation
# Pour les caractéristiques numériques
numeric_features_transformed = numeric_features

# Pour les caractéristiques catégorielles (après one-hot encoding)
categorical_features_transformed = []
for cat_feature in categorical_features:
    # Obtention des catégories uniques pour chaque caractéristique catégorielle
    unique_categories = df[cat_feature].unique()
    # Création des noms de colonnes après one-hot encoding (en excluant la première catégorie)
    for category in unique_categories[1:]:
        categorical_features_transformed.append(f"{cat_feature}_{category}")

# Combinaison des noms de caractéristiques
all_features_transformed = numeric_features_transformed + categorical_features_transformed

# Vérification de la longueur
print(f"Nombre de caractéristiques après transformation: {len(all_features_transformed)}")
print(f"Forme des données prétraitées: {X_train_preprocessed.shape[1]}")

# Ajustement si nécessaire (si les dimensions ne correspondent pas)
if len(all_features_transformed) != X_train_preprocessed.shape[1]:
    print("Attention: Le nombre de noms de caractéristiques ne correspond pas à la dimension des données prétraitées.")
    # Utilisation d'indices numériques si les noms ne correspondent pas
    all_features_transformed = [f"Feature_{i}" for i in range(X_train_preprocessed.shape[1])]

In [ ]:
# Extraction des importances des caractéristiques du modèle Random Forest
feature_importances = rf_clf.feature_importances_

# Création d'un DataFrame pour visualiser les importances
importance_df = pd.DataFrame({
    'Feature': all_features_transformed,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

# Affichage des 15 caractéristiques les plus importantes
top_15_features = importance_df.head(15)
top_15_features

In [ ]:
# Visualisation des 15 caractéristiques les plus importantes
fig = px.bar(top_15_features, 
             x='Importance', 
             y='Feature', 
             orientation='h',
             title='Top 15 des caractéristiques les plus importantes',
             color='Importance',
             color_continuous_scale='Viridis')

fig.update_layout(
    xaxis_title='Importance',
    yaxis_title='Caractéristique',
    width=800,
    height=500,
    yaxis={'categoryorder':'total ascending'}
)

fig.show()

## 8. Analyse des erreurs de prédiction

Analysons les cas où notre meilleur modèle (Random Forest) fait des erreurs de prédiction.

In [ ]:
# Prédictions du modèle Random Forest sur l'ensemble de validation
y_val_pred_rf = rf_clf.predict(X_val_preprocessed)

# Identification des erreurs
errors = y_val != y_val_pred_rf
X_val_errors = X_val[errors]
y_val_errors = y_val[errors]
y_val_pred_errors = y_val_pred_rf[errors]

print(f"Nombre d'erreurs: {errors.sum()} sur {len(y_val)} échantillons de validation ({errors.sum()/len(y_val)*100:.2f}%)")

In [ ]:
# Analyse des erreurs par classe
error_types = pd.DataFrame({
    'Réalité': y_val_errors,
    'Prédiction': y_val_pred_errors
})

# Conversion des valeurs numériques en étiquettes
error_types['Réalité'] = error_types['Réalité'].map({0: 'Non satisfait', 1: 'Satisfait'})
error_types['Prédiction'] = error_types['Prédiction'].map({0: 'Non satisfait', 1: 'Satisfait'})

# Comptage des types d'erreurs
error_counts = error_types.groupby(['Réalité', 'Prédiction']).size().reset_index(name='Nombre')
error_counts

## 9. Conclusion

Dans ce notebook, nous avons exploré plusieurs modèles de classification pour prédire la satisfaction des passagers :

1. **Modèle de base (Dummy Classifier)** : Ce modèle simple nous a fourni une référence de performance.
2. **Régression Logistique** : Un modèle linéaire qui a montré de bonnes performances.
3. **Arbre de Décision** : Un modèle plus complexe qui peut capturer des relations non linéaires.
4. **Random Forest** : Un ensemble d'arbres qui a généralement montré les meilleures performances.
5. **SVM** : Un modèle puissant pour la classification avec de bonnes performances.

Nous avons également analysé les caractéristiques les plus importantes pour la prédiction et examiné les types d'erreurs commises par notre meilleur modèle.

### Prochaines étapes

Pour améliorer davantage les performances des modèles, nous pourrions :
- Effectuer une optimisation des hyperparamètres pour affiner les modèles
- Explorer d'autres techniques de feature engineering
- Essayer des modèles plus avancés comme les réseaux de neurones ou le gradient boosting
- Traiter le déséquilibre des classes si nécessaire

Ces aspects seront abordés dans les prochains notebooks.